In [1]:
VERSION

v"1.8.1"

In [2]:
ENV["COLUMNS"] = 1000
ENV["LINES"] = 20;

In [3]:
# using Pkg
# Pkg.add("BenchmarkTools")
# Pkg.update("BenchmarkTools")
# Pkg.build("BenchmarkTools")

In [5]:
using DataFrames
using GLMakie
using JLD2
using BenchmarkTools

using Revise
using Astroshaper
import SPICE

In [6]:
meta_kernel = "/Users/masanorikanamaru/Dropbox/spice/hera/kernels/mk/hera_study_PO_EMA_2024_shapeViewer.tm"

SPICE.furnsh(meta_kernel)

In [7]:
et_start = SPICE.utc2et("2027-02-09T22:00:00")
et_end   = SPICE.utc2et("2027-02-11T22:00:00")
step     = 3600/20

et_range = et_start:step:et_end

@show et_range
@show length(et_range);

et_range = 8.554824691849957e8:180.0:8.556552691849957e8
length(et_range) = 961


In [8]:
# save_range = findall(et_range .> et_range[end] - 7.63262 * 3600)
save_range = 1:length(et_range)

@show save_range[begin]
@show save_range[end]
@show length(save_range);

save_range[begin] = 1
save_range[end] = 961
length(save_range) = 961


In [9]:
sun_d1 = spkpos("SUN",       et_range, "DIDYMOS_FIXED",   "None", "DIDYMOS")
sun_d2 = spkpos("SUN",       et_range, "DIMORPHOS_FIXED", "None", "DIMORPHOS")
d1_d2  = spkpos("DIDYMOS",   et_range, "DIMORPHOS_FIXED", "None", "DIMORPHOS")
d2_d1  = spkpos("DIMORPHOS", et_range, "DIDYMOS_FIXED",   "None", "DIDYMOS");

In [10]:
D1_TO_D2 = pxform("DIDYMOS_FIXED",   "DIMORPHOS_FIXED", et_range)
D2_TO_D1 = pxform("DIMORPHOS_FIXED", "DIDYMOS_FIXED",   et_range)

D1_TO_J2000 = pxform("DIDYMOS_FIXED",   "J2000", et_range)
D2_TO_J2000 = pxform("DIMORPHOS_FIXED", "J2000", et_range);

In [11]:
SPICE.kclear()

In [12]:
fig = Figure()
ax = Axis3(fig[1, 1], aspect=:data)

# scatter!(0,0,0)
# scatter!(sun_d1.x, sun_d1.y, sun_d1.z, color=:orange)
# scatter!(d2_d1.x, d2_d1.y, d2_d1.z, color=:skyblue)
# scatter!(sun_d1.x, sun_d1.y, sun_d1.z, color=:orange)
# scatter!(d2_d1.x, d2_d1.y, d2_d1.z, color=:skyblue)

scatter!([r[1] for r in sun_d1], [r[2] for r in sun_d1], [r[3] for r in sun_d1], color=:orange, size=2)

display(fig)

GLMakie.Screen(...)

# 形状モデル Didymos & Dimorphos

In [15]:
shapedir = "/Users/masanorikanamaru/Dropbox/spice/hera/kernels/dsk"

shapepath1 = joinpath(shapedir, "g_50677mm_rad_obj_dida_0000n00000_v001.obj")
shapepath2 = joinpath(shapedir, "g_06650mm_rad_obj_didb_0000n00000_v001.obj")

shape1 = ShapeModel(shapepath1; scale=1000, find_visible_facets=true, save_shape=false)
shape2 = ShapeModel(shapepath2; scale=1000, find_visible_facets=true, save_shape=false);

In [16]:
println(shape1)
println(shape2)

Shape model
-----------
Nodes             : 1000
Faces             : 1996
Surface area      : 1.9587914739775863e6
Volume            : 2.485481753075182e8
Equivalent radius : 390.03836143582186
Center-of-Figure  : [2.484674354494825e-6, -2.0997384259416257e-6, 2.190089591373722e-6]
Inertia tensor    : 
    | Ixx Ixy Ixz |   [0.0, 0.0, 0.0]
    | Iyx Iyy Iyz | = [0.0, 0.0, 0.0]
    | Izx Izy Izz |   [0.0, 0.0, 0.0]

Shape model
-----------
Nodes             : 1538
Faces             : 3072
Surface area      : 53920.63065752515
Volume            : 1.1314114665400542e6
Equivalent radius : 64.64138537338498
Center-of-Figure  : [2.009646200056278e-16, 3.5852088209004e-16, -2.9131831316015805e-15]
Inertia tensor    : 
    | Ixx Ixy Ixz |   [0.0, 0.0, 0.0]
    | Iyx Iyy Iyz | = [0.0, 0.0, 0.0]
    | Izx Izy Izz |   [0.0, 0.0, 0.0]



# TPM only on Didymos

In [20]:
thermo_params = ThermoParams(  # [Michel+2016; Naidu+2020]
    A_B   = 0.059,  # Bolometric  Bond albedo
    A_TH  = 0.0,
    k     = 0.125,
    ρ     = 2170.,
    Cp    = 600.,
    ϵ     = 0.9,
    t_bgn = et_range[begin],
    t_end = et_range[end],
    Nt    = length(et_range),
    z_max = 0.6,
    Nz    = 41,
    P     = SPICE.convrt(DIDYMOS[:P], "hours", "seconds"),
)

Thermophysical parameters
-------------------------
A_B   : 0.059
A_TH  : 0.0
k     : 0.125
ρ     : 2170.0
Cp    : 600.0
ϵ     : 0.9
-------------------------
t_bgn : 8.554824691849957e8
t_bgn : 105180.37410616312 (Normalized by period P)
t_end : 8.556552691849957e8
t_end : 105201.61962468656 (Normalized by period P)
Nt    : 961
Δt    : 180.0000000000556
Δt    : 0.022130748461919817 (Normalized by period P)
-------------------------
z_max : 0.6
z_max : 6.057014348123621 (Normalized by skin depth l)
Nz    : 41
Δz    : 0.015
Δz    : 0.15142535870309054 (Normalized by skin depth l)
-------------------------
P     : 8133.4800000000005
l     : 0.09905870541414048
Γ     : 403.4228550788862
λ     : 0.07680491551461667
-------------------------


In [99]:
savepath = "tmp.jld2"
run_TPM!(shape1, et_range, sun_d1, thermo_params, savepath, save_range)

jldopen(savepath, "r+") do file
    file["D1_TO_J2000"] = D1_TO_J2000[save_range]
end;

In [95]:
draw(shape1; data=:temperature, colormap=:vik)

GLMakie.Screen(...)

In [96]:
data = load("tmp.jld2");

In [98]:
# data["shape"]
# data["et_range"]
# data["sun"]
# data["thermo_params"]
# data["surf_temps"]
# data["forces"]
# data["torques"]

# data["D1_TO_J2000"]

# TPM only on Dimorphos

In [101]:
thermo_params = ThermoParams(  # [Michel+2016; Naidu+2020]
    A_B   = 0.059,  # Bolometric  Bond albedo
    A_TH  = 0.0,
    k     = 0.125,
    ρ     = 2170.,
    Cp    = 600.,
    ϵ     = 0.9,
    t_bgn = et_range[begin],
    t_end = et_range[end],
    Nt    = length(et_range),
    z_max = 0.6,
    Nz    = 41,
    P     = SPICE.convrt(DIDYMOS[:P], "hours", "seconds"),
);

In [105]:
savepath = "tmp.jld2"
run_TPM!(shape2, et_range, sun_d2, thermo_params, savepath, save_range)

jldopen(savepath, "r+") do file
    file["D2_TO_J2000"] = D2_TO_J2000[save_range]
end;

In [111]:
draw(shape2; data=:temperature, colormap=:vik)

LoadError: You should not pass the `colormap` attribute to the colorbar when constructing it using an existing plot object. This attribute is copied from the plot object, and setting it from the colorbar will make the plot object and the colorbar go out of sync.

# TPM on Binary Asteroid